In [1]:
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from dgllife.utils import smiles_to_bigraph, CanonicalAtomFeaturizer, CanonicalBondFeaturizer
from model import *  # 수정된 Mymodel 클래스 포함
import dgl
from dgllife.utils import Meter
import numpy as np
import random
import torch
from pytorchtools import EarlyStopping
from itertools import product
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score, average_precision_score
import csv

# 데이터 로드 및 전처리
Count = 343
smiles = pd.read_csv('/data/home/dbswn0814/2025JCM/model/DCAMCP/mgl/smiles.csv')
bigraphs = []
node_featurizer = CanonicalAtomFeaturizer()
edge_featurizer = CanonicalBondFeaturizer(self_loop=True)
for smile in smiles['smiles']:
    bigraphs.append(
        smiles_to_bigraph(
            smile,
            add_self_loop=True,
            node_featurizer=node_featurizer,
            edge_featurizer=edge_featurizer,
            canonical_atom_order=False,
        ))
bigraphs = bigraphs[:Count]

fp = pd.read_csv('/data/home/dbswn0814/2025JCM/model/DCAMCP/mgl/CDKExt.csv')
# 'Name' 컬럼 삭제
fp = fp.drop(columns=['Name'], errors='ignore')  # 'Name' 컬럼이 없어도 에러 없이 진행

# 데이터의 첫 Count 개수만 선택
fp = fp.iloc[:Count, :]

# NumPy 배열을 리스트로 변환
fp = np.array(fp).tolist()

# 첫 번째 리스트의 길이 확인
first_list_length = len(fp[0])
print(f"첫 번째 리스트의 길이: {first_list_length}")

labels = smiles['Class'].tolist()[:Count]

# 랜덤 시드 설정 함수
def setup_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

# 커스텀 Dataset 정의
class Mydataset(Dataset):
    def __init__(self, train_features, train_labels, fp):
        self.x_data = train_features
        self.y_data = train_labels
        self.fp = fp
        self.len = len(train_labels)

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index], self.fp[index]

    def __len__(self):
        return self.len

    def get_collate_fn(self):
        def _collate(data):
            graphs, labels, fp = map(list, zip(*data))
            batched_graph = dgl.batch(graphs)
            return batched_graph, torch.tensor(labels), torch.tensor(fp)
        return _collate

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 모델 학습 및 검증 함수
def traink(model, X_train, y_train, z_train, X_val, y_val, z_val, BATCH_SIZE, learning_rate, weight_decay, dropout, TOTAL_EPOCHS):
    data = Mydataset(X_train, y_train, z_train)
    collate = data.get_collate_fn()
    train_loader = DataLoader(data, BATCH_SIZE, collate_fn=collate, shuffle=True)
    val_data = Mydataset(X_val, y_val, z_val)
    val_loader = DataLoader(val_data, BATCH_SIZE, collate_fn=collate, shuffle=False)
    model = model.to(device)

    criterion = CapsuleLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5)
    early_stopping = EarlyStopping(patience=10, verbose=True)

    losses = []
    val_losses = []
    train_acc = []
    val_acc = []
    SE = []
    SP = []
    AUC = []
    AUPR = []

    for epoch in range(TOTAL_EPOCHS):
        model.train()
        correct = 0
        for i, (features, labels, fp) in enumerate(train_loader):
            features = features.to(device)
            fp = fp.to(device)
            labels = torch.eye(2).index_select(dim=0, index=labels).to(device)

            optimizer.zero_grad()
            logits = model(features, fp)
            loss = criterion(logits, labels)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 5)
            optimizer.step()
            losses.append(loss.item())
            correct += torch.sum(
                torch.argmax(logits, dim=1) == torch.argmax(labels, dim=1)).item()
        accuracy = 100. * correct / len(X_train)
#         print('Epoch: {}, Loss: {:.5f}, Training set accuracy: {}/{} ({:.3f}%)'
#               .format(epoch + 1, loss.item(), correct, len(X_train), accuracy))
        train_acc.append(accuracy)

        # Validation
        model.eval()
        val_loss = 0
        correct = 0
        y_true_all = []
        y_score_all = []
        with torch.no_grad():
            for i, (features, labels, fp) in enumerate(val_loader):
                features = features.to(device)
                fp = fp.to(device)
                labels_1 = torch.eye(2).index_select(dim=0, index=labels).to(device)

                logits = model(features, fp)
                loss = criterion(logits, labels_1).item()
                val_loss += loss * len(labels_1)

                preds = torch.argmax(logits, dim=1)
                targets = torch.argmax(labels_1, dim=1)
                correct += torch.sum(preds == targets).item()

                y_true_all.extend(labels.cpu().numpy())
                y_score_all.extend(logits[:, 1].cpu().numpy())

        # 성능 지표 계산
        try:
            auc = roc_auc_score(y_true_all, y_score_all)
            aupr = average_precision_score(y_true_all, y_score_all)
        except ValueError:
            auc = 0
            aupr = 0

        accuracy = 100. * correct / len(X_val)
        avg_val_loss = val_loss / len(X_val)
#         print('Validation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)\n AUC: {:.3f}, AUPR: {:.3f}\n'.format(
#             avg_val_loss, correct, len(X_val), accuracy, auc, aupr))

        val_losses.append(avg_val_loss)
        AUC.append(auc)
        AUPR.append(aupr)
        val_acc.append(accuracy)

        scheduler.step(auc)

        early_stopping(auc, model)

        if early_stopping.early_stop:
            print("Early stopping")
            break

    return losses, val_losses, train_acc, val_acc, AUC, AUPR

# K-Fold 교차검증 함수 (Stratified 30-Fold)
def k_fold(k, X, y, z, learning_rate, weight_decay, dropout, batch_size, num_epochs=100):
    skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=9)#4
    fold_results = []

    for fold, (train_index, test_index) in enumerate(skf.split(X, y)):
        print('*' * 25, f'Fold {fold + 1}', '*' * 25)
        X_train_full = [X[i] for i in train_index]
        y_train_full = [y[i] for i in train_index]
        z_train_full = [z[i] for i in train_index]

        X_test = [X[i] for i in test_index]
        y_test = [y[i] for i in test_index]
        z_test = [z[i] for i in test_index]

        # Train 데이터를 Train과 Validation으로 분할 (10% Validation)
        X_train, X_val, y_train, y_val, z_train, z_val = train_test_split(
            X_train_full, y_train_full, z_train_full, test_size=0.1, random_state=2, stratify=y_train_full)

        # 모델 초기화 (dropout을 인자로 추가)
        graph = X_train[0]
        n_feats = graph.ndata["h"].shape[1]
        fp_dim = len(z_train[0])
        model = Mymodel(n_feats=n_feats, fp=fp_dim, dropout=dropout)  # Mymodel에 dropout 파라미터 추가

        # 모델 학습
        train_loss, val_loss, train_acc, val_acc, AUC, AUPR = traink(
            model, X_train, y_train, z_train, X_val, y_val, z_val,
            BATCH_SIZE=batch_size, learning_rate=learning_rate, weight_decay=weight_decay, dropout=dropout, TOTAL_EPOCHS=num_epochs
        )

        # 테스트 데이터 로딩
        test_data = Mydataset(X_test, y_test, z_test)
        test_loader = DataLoader(test_data, batch_size, collate_fn=test_data.get_collate_fn(), shuffle=False)

        # 테스트 평가
        model.eval()
        test_loss = 0
        correct = 0
        y_true_all = []
        y_score_all = []
        with torch.no_grad():
            for features, labels, fp in test_loader:
                features = features.to(device)
                fp = fp.to(device)
                labels_1 = torch.eye(2).index_select(dim=0, index=labels).to(device)

                logits = model(features, fp)
                loss = CapsuleLoss()(logits, labels_1).item()
                test_loss += loss * len(labels_1)

                preds = torch.argmax(logits, dim=1)
                targets = torch.argmax(labels_1, dim=1)
                correct += torch.sum(preds == targets).item()

                y_true_all.extend(labels.cpu().numpy())
                y_score_all.extend(logits[:, 1].cpu().numpy())

        # 성능 지표 계산
        try:
            auc = roc_auc_score(y_true_all, y_score_all)
            aupr = average_precision_score(y_true_all, y_score_all)
        except ValueError:
            auc = 0
            aupr = 0

        test_accuracy = 100. * correct / len(X_test)
        test_avg_loss = test_loss / len(X_test)
        print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)\n AUC: {:.3f}, AUPR: {:.3f}\n'.format(
            test_avg_loss, correct, len(X_test), test_accuracy, auc, aupr))

        # Fold 결과 저장
        fold_result = {
            'fold': fold + 1,
            'test_loss': test_avg_loss,
            'test_accuracy': test_accuracy,
            'test_AUROC': auc,
            'test_AUPR': aupr
        }
        fold_results.append(fold_result)

    # 전체 Fold 결과 출력
    test_losses = [fr['test_loss'] for fr in fold_results]
    test_accuracies = [fr['test_accuracy'] for fr in fold_results]
    test_auc = [fr['test_AUROC'] for fr in fold_results]
    test_aupr = [fr['test_AUPR'] for fr in fold_results]

    avg_loss = np.mean(test_losses)
    std_loss = np.std(test_losses)
    avg_acc = np.mean(test_accuracies)
    std_acc = np.std(test_accuracies)
    avg_auc = np.mean(test_auc)
    std_auc = np.std(test_auc)
    avg_aupr = np.mean(test_aupr)
    std_aupr = np.std(test_aupr)

    print('\n', '#' * 10, '최종 20-Fold 교차검증 결과', '#' * 10)
    print(f'Average Test Loss: {avg_loss:.4f} ± {std_loss:.4f}')
    print(f'Average Test Accuracy: {avg_acc:.3f}% ± {std_acc:.3f}%')
    print(f'Average Test AUROC: {avg_auc:.3f} ± {std_auc:.3f}')
    print(f'Average Test AUPR: {avg_aupr:.3f} ± {std_aupr:.3f}')

    # 결과를 CSV 파일로 저장
    csv_columns = ['fold', 'test_loss', 'test_accuracy', 'test_AUROC', 'test_AUPR']
    csv_file = "kfold_results_mgl.csv"
    try:
        with open(csv_file, 'w', newline='') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
            writer.writeheader()
            for data in fold_results:
                writer.writerow(data)
        print(f'Fold 결과가 {csv_file}에 저장되었습니다.')
    except IOError:
        print("I/O 에러 발생")

    # 전체 결과를 반환 (그리드 서치에서 활용 가능)
    overall_results = {
        'average_test_loss': avg_loss,
        'std_test_loss': std_loss,
        'average_test_accuracy': avg_acc,
        'std_test_accuracy': std_acc,
        'average_test_AUROC': avg_auc,
        'std_test_AUROC': std_auc,
        'average_test_AUPR': avg_aupr,
        'std_test_AUPR': std_aupr
    }

    return overall_results

# 그리드 서치 및 K-Fold 실행 함수
def grid_search(k, X, y, z, param_grid, batch_size=8, num_epochs=100):
    # 모든 하이퍼파라미터 조합 생성
    keys, values = zip(*param_grid.items())
    param_combinations = [dict(zip(keys, v)) for v in product(*values)]

    all_results = []

    for idx, params in enumerate(param_combinations):
        print('=' * 20, f'Grid Search {idx + 1}/{len(param_combinations)}', '=' * 20)
        learning_rate = params.get('learning_rate', 0.001)
        weight_decay = params.get('weight_decay', 0.0)
        dropout = params.get('dropout', 0.2)

        print(f"하이퍼파라미터 조합: learning_rate={learning_rate}, weight_decay={weight_decay}, dropout={dropout}")

        # 각 하이퍼파라미터 조합에 대해 Stratified K-Fold 교차검증 실행
        fold_metrics = k_fold(
            k, X, y, z,
            learning_rate=learning_rate,
            weight_decay=weight_decay,
            dropout=dropout,
            batch_size=batch_size,
            num_epochs=num_epochs
        )

        # 전체 결과와 하이퍼파라미터 조합을 함께 저장
        result = {
            'learning_rate': learning_rate,
            'weight_decay': weight_decay,
            'dropout': dropout,
            'average_test_loss': fold_metrics['average_test_loss'],
            'std_test_loss': fold_metrics['std_test_loss'],
            'average_test_accuracy': fold_metrics['average_test_accuracy'],
            'std_test_accuracy': fold_metrics['std_test_accuracy'],
            'average_test_AUROC': fold_metrics['average_test_AUROC'],
            'std_test_AUROC': fold_metrics['std_test_AUROC'],
            'average_test_AUPR': fold_metrics['average_test_AUPR'],
            'std_test_AUPR': fold_metrics['std_test_AUPR']
        }
        all_results.append(result)

    # 그리드 서치 전체 결과를 CSV 파일로 저장
    grid_csv_columns = [
        'learning_rate', 'weight_decay', 'dropout',
        'average_test_loss', 'std_test_loss',
        'average_test_accuracy', 'std_test_accuracy',
        'average_test_AUROC', 'std_test_AUROC',
        'average_test_AUPR', 'std_test_AUPR'
    ]
    grid_csv_file = "grid_search_results_mgl.csv"
    try:
        with open(grid_csv_file, 'w', newline='') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=grid_csv_columns)
            writer.writeheader()
            for data in all_results:
                writer.writerow(data)
        print(f'그리드 서치 결과가 {grid_csv_file}에 저장되었습니다.')
    except IOError:
        print("I/O 에러 발생")

    print('그리드 서치가 완료되었습니다.')
    return

if __name__ == '__main__':
    setup_seed(4)


    param_grid = {
        'learning_rate': [0.001, 0.0001],
        'weight_decay': [1e-06, 1e-05, 1e-04],
        'dropout': [0.2]  
    }


    grid_search(
        k=20,
        X=bigraphs,
        y=labels,
        z=fp,
        param_grid=param_grid,
        batch_size=8,  
        num_epochs=1000   
    )


첫 번째 리스트의 길이: 1024
==================== Grid Search 1/6 ====================
하이퍼파라미터 조합: learning_rate=0.001, weight_decay=1e-06, dropout=0.2
************************* Fold 1 *************************
meter decreased (inf --> 0.654412).  Saving model ...
meter decreased (0.654412 --> 0.750000).  Saving model ...
EarlyStopping counter: 1 out of 10
EarlyStopping counter: 2 out of 10
EarlyStopping counter: 3 out of 10
meter decreased (0.750000 --> 0.834559).  Saving model ...
EarlyStopping counter: 1 out of 10
EarlyStopping counter: 2 out of 10
EarlyStopping counter: 3 out of 10
EarlyStopping counter: 4 out of 10
EarlyStopping counter: 5 out of 10
EarlyStopping counter: 6 out of 10
EarlyStopping counter: 7 out of 10
EarlyStopping counter: 8 out of 10
EarlyStopping counter: 9 out of 10
EarlyStopping counter: 10 out of 10
Early stopping
Test set: Average loss: 0.2763, Accuracy: 13/18 (72.222%)
 AUC: 0.753, AUPR: 0.783

************************* Fold 2 *************************
meter decreas

EarlyStopping counter: 8 out of 10
EarlyStopping counter: 9 out of 10
meter decreased (0.808824 --> 0.834559).  Saving model ...
EarlyStopping counter: 1 out of 10
EarlyStopping counter: 2 out of 10
EarlyStopping counter: 3 out of 10
EarlyStopping counter: 4 out of 10
EarlyStopping counter: 5 out of 10
EarlyStopping counter: 6 out of 10
EarlyStopping counter: 7 out of 10
EarlyStopping counter: 8 out of 10
EarlyStopping counter: 9 out of 10
EarlyStopping counter: 10 out of 10
Early stopping
Test set: Average loss: 0.3305, Accuracy: 13/17 (76.471%)
 AUC: 0.750, AUPR: 0.677

************************* Fold 10 *************************
meter decreased (inf --> 0.661765).  Saving model ...
meter decreased (0.661765 --> 0.727941).  Saving model ...
EarlyStopping counter: 1 out of 10
meter decreased (0.727941 --> 0.738971).  Saving model ...
EarlyStopping counter: 1 out of 10
EarlyStopping counter: 2 out of 10
meter decreased (0.738971 --> 0.768382).  Saving model ...
EarlyStopping counter: 1 

EarlyStopping counter: 6 out of 10
EarlyStopping counter: 7 out of 10
EarlyStopping counter: 8 out of 10
meter decreased (0.797794 --> 0.797794).  Saving model ...
EarlyStopping counter: 1 out of 10
EarlyStopping counter: 2 out of 10
EarlyStopping counter: 3 out of 10
EarlyStopping counter: 4 out of 10
EarlyStopping counter: 5 out of 10
EarlyStopping counter: 6 out of 10
EarlyStopping counter: 7 out of 10
EarlyStopping counter: 8 out of 10
EarlyStopping counter: 9 out of 10
EarlyStopping counter: 10 out of 10
Early stopping
Test set: Average loss: 0.2379, Accuracy: 14/17 (82.353%)
 AUC: 0.806, AUPR: 0.792

************************* Fold 18 *************************
meter decreased (inf --> 0.628676).  Saving model ...
EarlyStopping counter: 1 out of 10
meter decreased (0.628676 --> 0.650735).  Saving model ...
EarlyStopping counter: 1 out of 10
EarlyStopping counter: 2 out of 10
meter decreased (0.650735 --> 0.661765).  Saving model ...
EarlyStopping counter: 1 out of 10
EarlyStopping 

EarlyStopping counter: 4 out of 10
meter decreased (0.764706 --> 0.786765).  Saving model ...
EarlyStopping counter: 1 out of 10
EarlyStopping counter: 2 out of 10
EarlyStopping counter: 3 out of 10
EarlyStopping counter: 4 out of 10
EarlyStopping counter: 5 out of 10
meter decreased (0.786765 --> 0.790441).  Saving model ...
meter decreased (0.790441 --> 0.845588).  Saving model ...
EarlyStopping counter: 1 out of 10
EarlyStopping counter: 2 out of 10
EarlyStopping counter: 3 out of 10
EarlyStopping counter: 4 out of 10
EarlyStopping counter: 5 out of 10
EarlyStopping counter: 6 out of 10
EarlyStopping counter: 7 out of 10
EarlyStopping counter: 8 out of 10
EarlyStopping counter: 9 out of 10
EarlyStopping counter: 10 out of 10
Early stopping
Test set: Average loss: 0.2893, Accuracy: 14/17 (82.353%)
 AUC: 0.889, AUPR: 0.879

************************* Fold 6 *************************
meter decreased (inf --> 0.687500).  Saving model ...
EarlyStopping counter: 1 out of 10
EarlyStopping c

meter decreased (inf --> 0.801471).  Saving model ...
EarlyStopping counter: 1 out of 10
meter decreased (0.801471 --> 0.808824).  Saving model ...
EarlyStopping counter: 1 out of 10
meter decreased (0.808824 --> 0.823529).  Saving model ...
meter decreased (0.823529 --> 0.860294).  Saving model ...
meter decreased (0.860294 --> 0.941176).  Saving model ...
EarlyStopping counter: 1 out of 10
EarlyStopping counter: 2 out of 10
EarlyStopping counter: 3 out of 10
EarlyStopping counter: 4 out of 10
EarlyStopping counter: 5 out of 10
EarlyStopping counter: 6 out of 10
EarlyStopping counter: 7 out of 10
EarlyStopping counter: 8 out of 10
EarlyStopping counter: 9 out of 10
EarlyStopping counter: 10 out of 10
Early stopping
Test set: Average loss: 0.2673, Accuracy: 14/17 (82.353%)
 AUC: 0.903, AUPR: 0.912

************************* Fold 15 *************************
meter decreased (inf --> 0.768382).  Saving model ...
meter decreased (0.768382 --> 0.779412).  Saving model ...
meter decreased (

...

EarlyStopping counter: 1 out of 10
EarlyStopping counter: 2 out of 10
EarlyStopping counter: 3 out of 10
meter decreased (0.908088 --> 0.911765).  Saving model ...
EarlyStopping counter: 1 out of 10
EarlyStopping counter: 2 out of 10
EarlyStopping counter: 3 out of 10
EarlyStopping counter: 4 out of 10
EarlyStopping counter: 5 out of 10
EarlyStopping counter: 6 out of 10
EarlyStopping counter: 7 out of 10
EarlyStopping counter: 8 out of 10
EarlyStopping counter: 9 out of 10
EarlyStopping counter: 10 out of 10
Early stopping
Test set: Average loss: 0.3441, Accuracy: 15/17 (88.235%)
 AUC: 0.986, AUPR: 0.986

************************* Fold 8 *************************
meter decreased (inf --> 0.632353).  Saving model ...
meter decreased (0.632353 --> 0.797794).  Saving model ...
EarlyStopping counter: 1 out of 10
EarlyStopping counter: 2 out of 10
EarlyStopping counter: 3 out of 10
EarlyStopping counter: 4 out of 10
EarlyStopping counter: 5 out of 10
EarlyStopping counter: 6 out of 10
Earl

EarlyStopping counter: 1 out of 10
EarlyStopping counter: 2 out of 10
EarlyStopping counter: 3 out of 10
EarlyStopping counter: 4 out of 10
EarlyStopping counter: 5 out of 10
meter decreased (0.897059 --> 0.900735).  Saving model ...
EarlyStopping counter: 1 out of 10
EarlyStopping counter: 2 out of 10
EarlyStopping counter: 3 out of 10
EarlyStopping counter: 4 out of 10
EarlyStopping counter: 5 out of 10
EarlyStopping counter: 6 out of 10
EarlyStopping counter: 7 out of 10
EarlyStopping counter: 8 out of 10
EarlyStopping counter: 9 out of 10
EarlyStopping counter: 10 out of 10
Early stopping
Test set: Average loss: 0.3725, Accuracy: 13/17 (76.471%)
 AUC: 0.889, AUPR: 0.939

************************* Fold 15 *************************
meter decreased (inf --> 0.602941).  Saving model ...
meter decreased (0.602941 --> 0.757353).  Saving model ...
EarlyStopping counter: 1 out of 10
EarlyStopping counter: 2 out of 10
EarlyStopping counter: 3 out of 10
meter decreased (0.757353 --> 0.761029

EarlyStopping counter: 6 out of 10
EarlyStopping counter: 7 out of 10
EarlyStopping counter: 8 out of 10
EarlyStopping counter: 9 out of 10
EarlyStopping counter: 10 out of 10
Early stopping
Test set: Average loss: 0.3991, Accuracy: 15/18 (83.333%)
 AUC: 0.790, AUPR: 0.866

************************* Fold 2 *************************
meter decreased (inf --> 0.518382).  Saving model ...
meter decreased (0.518382 --> 0.647059).  Saving model ...
meter decreased (0.647059 --> 0.783088).  Saving model ...
EarlyStopping counter: 1 out of 10
EarlyStopping counter: 2 out of 10
EarlyStopping counter: 3 out of 10
meter decreased (0.783088 --> 0.819853).  Saving model ...
meter decreased (0.819853 --> 0.834559).  Saving model ...
EarlyStopping counter: 1 out of 10
EarlyStopping counter: 2 out of 10
meter decreased (0.834559 --> 0.878676).  Saving model ...
EarlyStopping counter: 1 out of 10
EarlyStopping counter: 2 out of 10
EarlyStopping counter: 3 out of 10
EarlyStopping counter: 4 out of 10
Ea

EarlyStopping counter: 10 out of 10
Early stopping
Test set: Average loss: 0.4089, Accuracy: 10/17 (58.824%)
 AUC: 0.597, AUPR: 0.601

************************* Fold 10 *************************
meter decreased (inf --> 0.595588).  Saving model ...
meter decreased (0.595588 --> 0.672794).  Saving model ...
EarlyStopping counter: 1 out of 10
meter decreased (0.672794 --> 0.705882).  Saving model ...
EarlyStopping counter: 1 out of 10
EarlyStopping counter: 2 out of 10
EarlyStopping counter: 3 out of 10
meter decreased (0.705882 --> 0.735294).  Saving model ...
meter decreased (0.735294 --> 0.750000).  Saving model ...
EarlyStopping counter: 1 out of 10
meter decreased (0.750000 --> 0.753676).  Saving model ...
EarlyStopping counter: 1 out of 10
EarlyStopping counter: 2 out of 10
EarlyStopping counter: 3 out of 10
EarlyStopping counter: 4 out of 10
EarlyStopping counter: 5 out of 10
EarlyStopping counter: 6 out of 10
EarlyStopping counter: 7 out of 10
EarlyStopping counter: 8 out of 10
E

EarlyStopping counter: 4 out of 10
meter decreased (0.937500 --> 0.937500).  Saving model ...
EarlyStopping counter: 1 out of 10
EarlyStopping counter: 2 out of 10
EarlyStopping counter: 3 out of 10
meter decreased (0.937500 --> 0.937500).  Saving model ...
EarlyStopping counter: 1 out of 10
EarlyStopping counter: 2 out of 10
EarlyStopping counter: 3 out of 10
EarlyStopping counter: 4 out of 10
EarlyStopping counter: 5 out of 10
meter decreased (0.937500 --> 0.937500).  Saving model ...
EarlyStopping counter: 1 out of 10
EarlyStopping counter: 2 out of 10
EarlyStopping counter: 3 out of 10
EarlyStopping counter: 4 out of 10
EarlyStopping counter: 5 out of 10
EarlyStopping counter: 6 out of 10
EarlyStopping counter: 7 out of 10
EarlyStopping counter: 8 out of 10
EarlyStopping counter: 9 out of 10
EarlyStopping counter: 10 out of 10
Early stopping
Test set: Average loss: 0.3951, Accuracy: 13/17 (76.471%)
 AUC: 0.833, AUPR: 0.874

************************* Fold 15 ************************